In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import matplotlib.patches as patches
import numpy as np 
from scipy import interp
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pylab as plt
from sklearn.metrics import roc_curve,auc
import statistics
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("cleve.mod", sep = "\s+", header=None)
df.columns = ["age", "sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","class","target"]

#Removing empty values
df.isna().sum()

#Labelling all the colums in the dataset
l1 = LabelEncoder()
df = df.apply(l1.fit_transform)

#Removing outliers using Inter Quartile Range
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]


x = df.loc[:, df.columns != 'target']
y = df.iloc[:,-1:] 


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.33, random_state = 0)
print(X_train.shape)

(130, 14)


In [ ]:
# Feature Scaling because yes we don't want one independent variable dominating the other and it makes computations easy
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from keras.models import Sequential
from keras.layers import Dense
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu', input_dim = 14))

# Adding the second hidden layer
classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN | means applying SGD on the whole ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, Y_train, batch_size = 10, epochs = 100,verbose = 0)

score, acc = classifier.evaluate(X_train, Y_train,
                            batch_size=10)
print('Train score:', score)
print('Train accuracy:', acc)
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

print('*'*20)
score, acc = classifier.evaluate(X_test, Y_test,
                            batch_size=10)
print('Test score:', score)
print('Test accuracy:', acc)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)

13/13 [==============================] - 0s 831us/step - loss: -1724.3536 - accuracy: 0.3538
Train score: -1724.3536376953125
Train accuracy: 0.35384616255760193
********************
7/7 [==============================] - 0s 2ms/step - loss: -1836.9540 - accuracy: 0.4462
Test score: -1836.9539794921875
Test accuracy: 0.446153849363327


In [24]:
import matplotlib.pylab as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import pandas as pd 
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("both_covid_data.csv")
x = df.loc[:, df.columns != 'Class']
y = df.iloc[:,-1:]

# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# evaluate each model in turn
results = []
names = []
scoring="accuracy"
for name, model in models:
    kfold = model_selection.RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)
    cv_results = model_selection.cross_val_score(model, x, y, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    accuracy = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(accuracy)


LR: 0.875910 (0.017727)
LDA: 0.838215 (0.023684)
KNN: 0.961853 (0.011452)
CART: 0.980623 (0.008266)
NB: 0.665304 (0.029979)
SVM: 0.965815 (0.011914)
